In [ ]:
#from __future__ import unicode_literals
%reload_ext autoreload
%autoreload 2

import os
from os.path import expanduser
home = expanduser("~")
import sys
sys.path.append(home + '/workspace/networkqit')
import numpy as np
import pandas as pd
import scipy
from scipy.io import loadmat
import networkqit as nq
import networkx as nx
import bct
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pickle
import re
# Additional utility functions
#from FrontiersAnalysis import data_grid_plot, analyze_forcellini_data, forcellini_plotting, compute_step, collect_step
import gzip
import seaborn as sns

In [ ]:
def percolation_analysis(W,nsteps=10):
    wmin = W[np.nonzero(W)].min()
    wmax = W[np.nonzero(W)].max()
    allw = np.unique(W.flatten())
    all_comps = []
    all_comps_size = []
    allt = np.linspace(wmin,wmax,nsteps)
    giant_size = []
    for t in allt:
        comps, comp_size = bct.get_components(bct.threshold_absolute(W,t))
        all_comps.append(comps)
        all_comps_size.append(len(comp_size))
        giant_size.append(np.max(comp_size))
    return allt,np.array(all_comps_size), np.array(giant_size)

In [ ]:
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')
allt,comps,giantsize = percolation_analysis(W, 50)

In [ ]:
plt.plot(allt,giantsize)
plt.vlines(x=allt[np.array(giantsize)==638][-1],ymin=0,ymax=638,color='k', alpha=0.6)
plt.vlines(x=allt[np.array(giantsize)==637][-1],ymin=0,ymax=638,color='k', alpha=0.6)
plt.grid()

allt[np.array(giantsize)==637][-1], allt[np.array(giantsize)==634][-1]

In [ ]:
plt.style.use('default')
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')[0:64,0:64]
beta_range = np.logspace(-3,3,100)

allt = np.linspace(0.4, 0.9, 12)
fig, ax = plt.subplots(ncols=4,nrows=3,figsize=(16,12))
ax = ax.flatten()

from tqdm import tqdm
for i,t in enumerate(allt):
    M = nq.CWTECM(N=len(W), threshold=t)
    G = bct.threshold_absolute(W,t)
    sol = M.fit(G=G, method='MLE',verbose=2,maxiter=2000)
    sol = M.fit(G=G,x0=sol['x'], method='saddle_point', verbose=1, maxiter=10)
    
    Amodel = []
    while len(Amodel) < 1:
        X = np.squeeze(M.sample_adjacency(sol['x'], batch_size=1)[0,:,:])
        #if bct.number_of_components(X) == 1:
        Amodel.append(X)
    Sdata = nq.batch_compute_vonneumann_entropy(L=nq.graph_laplacian(G), beta_range=beta_range)
    ax[i].semilogx(beta_range,Sdata,label='Data')
    #for j in range(len(Amodel)):
    #    Smodel = nq.batch_compute_vonneumann_entropy(L=nq.graph_laplacian(np.squeeze(M.sample_adjacency(sol['x']))),beta_range=beta_range)
    #    ax[i].semilogx(beta_range, Smodel, label='Model (sampled)')
    #    plt.gca().set_prop_cycle(None)
    SmodelExpected = nq.batch_compute_vonneumann_entropy(L=nq.graph_laplacian(M.expected_weighted_adjacency(sol['x'])),beta_range=beta_range)
    ax[i].semilogx(beta_range, SmodelExpected, label='Model (expected)')
    ax[i].legend(loc='best')
    ax[i].set_title('threshold=%.3f' % t)
    ax[i].set_xlabel('$\\beta$')
    ax[i].set_ylabel('$S(\\beta)$')    
plt.tight_layout()
plt.savefig('threshold_cwtecm.pdf',bbox='tight',dpi=200)

In [ ]:
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')
beta_range = np.logspace(-4,4,100)

allt = np.linspace(0.4, 0.9, 11)
fig, ax = plt.subplots(ncols=4,nrows=3,figsize=(16,12))
ax = ax.flatten()

for i,t in enumerate(allt):
    M = nq.CWTECM(N=len(W), threshold=t)
    G = bct.threshold_absolute(W,t)
    sol = M.fit(G=G, method='MLE', ftol=1E-3)
    Amodel = []
    count=0
    while len(Amodel) < 10:
        X = np.squeeze(M.sample_adjacency(sol['x'], batch_size=1)[0,:,:])
        if count>50:
            print('Too many iterations')
        if bct.number_of_components(X) ==  bct.number_of_components(G):
            Amodel.append(X)

    for j in range(len(Amodel)):
        dkl = nq.batch_beta_relative_entropy(Lobs=nq.graph_laplacian(G),Lmodel=nq.graph_laplacian(np.array(Amodel)), beta_range=beta_range)
    
    #ax[i].semilogx(beta_range, dkl, label='dkl')
    ax[i].semilogx(beta_range, dkl/(beta_range*np.sum(G)), label='dkl/beta$W_{tot}$')
    
    #ax[i].legend(['$S(\rho \| \sigma)$','$S(\rho \| \sigma)/\beta W_{tot}$'])
    ax[i].set_title('threshold=%.3f' % t)
    ax[i].set_xlabel('$\\beta$')
    ax[i].set_ylabel('$S(\\beta)$')
    
plt.tight_layout()
plt.savefig('dkl_normalized_crossley_groupaverage_mat_thresholded.pdf',bbox='tight')

In [ ]:
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')[10:30,10:30]
allt = np.linspace(0.4, 0.8, 12)
beta_range=np.logspace(-3,3, 100)

def compute_dkl(W,t,beta_range):
    G = bct.threshold_absolute(W,t)
    M = nq.CWTECM(N=len(W), threshold=G[np.nonzero(G)].min())
    sol = M.fit(G=G,method='MLE',ftol=1E-3)
    Amodel = []
    while len(Amodel) < 10:
        X = np.squeeze(M.sample_adjacency(sol['x'], batch_size=1)[0,:,:])
        if bct.number_of_components(X) ==  bct.number_of_components(G):
            Amodel.append(X)
    dkl = nq.batch_beta_relative_entropy(Lobs=nq.graph_laplacian(G),
                                         Lmodel=nq.graph_laplacian(np.array(Amodel)),
                                         beta_range=beta_range)
    return dkl
df = pd.DataFrame(data=np.array([compute_dkl(W,t,np.logspace(-3,3,100)) for t in allt]),columns=['dkl']).set_index(allt)
#df['dklnorm'] = [df['dkl']/(bct.threshold_absolute(W,t).sum()*beta_range) for t in allt ]

In [ ]:
plt.style.use('default')
W = np.loadtxt('/home/carlo/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj') #[0:25,0:64]
beta_range = np.logspace(-3,3,100)

t=0.6
G = bct.threshold_absolute(W,t)
M = nq.CWTECM(N=len(W), threshold=G[np.nonzero(G)].min() )
sol = M.fit(G=G, method='MLE',verbose=2, maxiter=1000)
#sol = M.fit(G=G,x0=sol['x'], verbose=2, method='saddle_point')

In [ ]:
Sdata = nq.batch_compute_vonneumann_entropy(L=nq.graph_laplacian(G), beta_range=beta_range)
plt.semilogx(beta_range,Sdata,label='Data')
X=M.expected_weighted_adjacency(sol['x'])
SmodelExpected = nq.batch_compute_vonneumann_entropy(L=nq.graph_laplacian(X),beta_range=beta_range)

plt.semilogx(beta_range, SmodelExpected, label='Model (expected)')
plt.legend(loc='best')
#for _ in range(50):
#    X = np.squeeze(M.sample_adjacency(sol['x']))
#    SmodelSampled = nq.batch_compute_vonneumann_entropy(L=nq.graph_laplacian(X),beta_range=beta_range)
#    plt.semilogx(beta_range,SmodelSampled,label='Model (sampled)',color='grey',alpha=0.2)
plt.title('threshold=%.3f' % t)
plt.xlabel('$\\beta$')
plt.ylabel('$S(\\beta)$')    
plt.tight_layout()
plt.savefig('threshold_cwtecm_full_thresh_0.6.pdf',bbox='tight',dpi=200)

In [ ]:
nq.plot_mle(G, M.expected_adjacency(sol['x']), M.expected_weighted_adjacency(sol['x']))

In [ ]:
dkl = nq.batch_beta_relative_entropy(Lobs=nq.graph_laplacian(G), Lmodel=nq.graph_laplacian(np.expand_dims(M.expected_weighted_adjacency(sol['x']),0)), beta_range=beta_range)
plt.semilogx(beta_range, dkl, label='$S(\\rho \| \\sigma)$')
#dkl = nq.batch_beta_relative_entropy(Lobs=nq.graph_laplacian(G), Lmodel=nq.graph_laplacian(M.sample_adjacency(sol['x'],100)), beta_range=beta_range)
#plt.semilogx(beta_range, dkl, label='$S(\\rho \| \\sigma)$')
plt.xlabel('$\\beta$')
plt.ylabel('$S(\\rho \| \\sigma)$')
plt.tight_layout()
plt.savefig('dkl_threshold_05.pdf',bbox='tight',dpi=200)

In [ ]:
def andd_bu(G):
    A = (G>0).astype(float)
    k = A.sum(0)
    #k[k<=0]=1
    return np.einsum('ij,i->j',G,k)/k

def andd_wu(G):
    A = (G>0).astype(float)
    s = G.sum(0)
    k = A.sum(0)
    #k[k<=0]=1
    return np.einsum('ij,i->j',G,s)/k

In [ ]:
plt.plot(andd_wu(G),andd_wu(np.squeeze(M.sample_adjacency(sol['x']))),'.r',markersize=1)
plt.plot(np.linspace(0.1,200,10),np.linspace(0.1,200,10),'-k', linewidth=0.5)
plt.xlabel('s^{nn}')
plt.ylabel('$\\langle s^{nn}\\rangle$')
plt.grid()
plt.axis('square')
plt.title('cTWECM')
#plt.xlim([-1,2])
#plt.ylim([-1,2])
#plt.savefig('andd_wu_groupaverage_n_638_threshold_06.pdf',bbox='tight')

In [ ]:
plt.plot(andd_bu(G>0),andd_bu(np.squeeze(M.sample_adjacency(sol['x']))>0),'.r',markersize=0.5)
plt.plot(np.linspace(0,200,10),np.linspace(0,200,10),'-k',linewidth=0.5)
plt.xlabel('$k^{nn}$')
plt.ylabel('$\\langle k^{nn}\\rangle$')
#plt.legend(loc='best')
plt.axis('square')
plt.title('cTWECM')
#plt.xlim([0,1])
#plt.ylim([0,1])
#plt.savefig('andd_wu_groupaverage_n_638_threshold_06.pdf',bbox='tight')

In [ ]:
Mb = nq.UECM3(N=len(G))
Gu = np.round(G*10)
soluecm3 = Mb.fit(Gu, method='MLE')
soluemc3 = Mb.fit(Gu,method='saddle_point',x0=soluecm3['x'],verbose=2)

In [ ]:
nq.plot_mle(np.round(G*100), Mb.expected_adjacency(soluecm3['x']),Mb.expected_weighted_adjacency(soluecm3['x']))

In [ ]:
plt.loglog(andd_wu(np.round(G*100)),andd_wu(np.squeeze(Mb.sample_adjacency(soluecm3['x']))),'.r',markersize=1)
plt.loglog(np.linspace(0.1,1E6,10),np.linspace(0.1,1E6,10),'-k',linewidth=0.5)
plt.xlabel('s^{nn}')
plt.ylabel('$\\langle s^{nn}\\rangle$')
plt.grid()
plt.axis('square')
plt.title('ECM')
#plt.xlim([-1,2])
#plt.ylim([-1,2])
#plt.savefig('andd_wu_groupaverage_n_638_threshold_06.pdf',bbox='tight')